<a href="https://colab.research.google.com/github/JamesPeralta/Machine-Learning-Algorithms/blob/master/Multi-Output%20Models/Social%20Media%20Person%20Predictor/Person_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-output models
### We can use the functional API to build models with multiple outputs (or multiple heads)

### Problem: Create a network that takes as input a series of social media posts from a single anonymous person and tries to predict attributes of that person, such as age, gender, and income level

In [1]:
from keras import layers
from keras import Input
from keras.models import Model

Using TensorFlow backend.


In [0]:
vocabulary_size = 50000
num_income_groups = 10

### Define Input Layer

In [0]:
posts_input = Input(shape=(None,), dtype='int32', name='posts')

### Define the Embedding Layer (Layer 1)

In [5]:
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)

Instructions for updating:
Colocations handled automatically by placer.


### Define the Convnet (Layer 2)

In [0]:
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)

### Define the Dense Nets (Layer 3)

In [0]:
# Global Dense Network
x = layers.Dense(128, activation='relu')(x)

# L3_D1
age_prediction = layers.Dense(1, name='age')(x)

# L3_D2
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)

# L3_D3
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

### Create the final model

In [8]:
model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    32000128    embedding_1[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (

In [0]:
model.compile(optimizer='rmsprop', 
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'}, 
              loss_weights={'age': 0.25,
                            'income': 1., 
                            'gender': 10.})

In [0]:
model.fit(posts, 
          {'age': age_targets,
           'income': income_targets,
           'gender': gender_targets},
           epochs=10, 
           batch_size=64)